In [1]:
import re
import pandas as pd
import mysql.connector
import numpy as np
from datetime import datetime

from utils_text import strip_accents

In [2]:
def clean_text(string: str) -> str:
    string = string.lower()
    string = strip_accents(string)
    string = string.strip()
    
    return string 

def to_datetime(string: str) -> str:
    return datetime.strptime(string, '%d/%m/%Y')

# Dataset

## df_viagem

In [3]:
df_viagem = pd.read_csv('./dataset/2020_Viagem.csv', delimiter=";", encoding='latin1')

In [4]:
df_viagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323503 entries, 0 to 323502
Data columns (total 21 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   Identificador do processo de viagem  323503 non-null  int64 
 1   Número da Proposta (PCDP)            323503 non-null  object
 2   Situação                             323503 non-null  object
 3   Viagem Urgente                       323503 non-null  object
 4   Justificativa Urgência Viagem        323442 non-null  object
 5   Código do órgão superior             323503 non-null  int64 
 6   Nome do órgão superior               323503 non-null  object
 7   Código órgão solicitante             323503 non-null  int64 
 8   Nome órgão solicitante               323503 non-null  object
 9   CPF viajante                         323475 non-null  object
 10  Nome                                 323503 non-null  object
 11  Cargo                     

In [5]:
df_viagem = df_viagem.drop('Motivo', axis=1)
df_viagem = df_viagem.drop('Valor outros gastos', axis=1)
df_viagem = df_viagem.drop('Valor passagens', axis=1)
df_viagem = df_viagem.drop('Valor diárias', axis=1)
df_viagem = df_viagem.drop('Destinos', axis=1)
df_viagem = df_viagem.drop('Justificativa Urgência Viagem', axis=1)
df_viagem = df_viagem.drop('Código órgão solicitante', axis=1)
df_viagem = df_viagem.drop('Nome órgão solicitante', axis=1)
df_viagem = df_viagem.drop('Número da Proposta (PCDP)', axis=1)
df_viagem = df_viagem.reset_index(drop=True)

In [6]:
df_viagem = df_viagem.rename(columns={'Identificador do processo de viagem': 'id_viagem'})

In [7]:
df_viagem = df_viagem.replace({np.nan: "Sem informação"})

In [8]:
# Tratamento de texto: acentos e simbolos, lower case e strip
for column in df_viagem.columns:
    if df_viagem[column].dtypes == 'object':
        df_viagem[column] = df_viagem[column].apply(clean_text)

In [9]:
df_viagem['Período - Data de início'] = df_viagem['Período - Data de início'].apply(to_datetime)
df_viagem['Período - Data de fim'] = df_viagem['Período - Data de fim'].apply(to_datetime)

In [10]:
df_viagem = df_viagem.replace({"nao informado": "sem informacao"})
df_viagem = df_viagem.replace({"-1": "sem informacao"})
df_viagem = df_viagem.replace({-1: "sem informacao"})
df_viagem['Código do órgão superior'] = df_viagem['Código do órgão superior'].replace({"sem informacao": -1})

In [11]:
df_remove = df_viagem[df_viagem['CPF viajante'].str.contains(r'ID.*|\*\*\*.000.000-\*\*')]
lista_id_remove = list(df_remove['id_viagem'])
df_viagem = df_viagem.drop(index=df_remove.index)
df_viagem = df_viagem.reset_index(drop=True)

In [12]:
df_viagem.index += 1

In [13]:
df_viagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322017 entries, 1 to 322017
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   id_viagem                 322017 non-null  object        
 1   Situação                  322017 non-null  object        
 2   Viagem Urgente            322017 non-null  object        
 3   Código do órgão superior  322017 non-null  int64         
 4   Nome do órgão superior    322017 non-null  object        
 5   CPF viajante              322017 non-null  object        
 6   Nome                      322017 non-null  object        
 7   Cargo                     322017 non-null  object        
 8   Função                    322017 non-null  object        
 9   Descrição Função          322017 non-null  object        
 10  Período - Data de início  322017 non-null  datetime64[ns]
 11  Período - Data de fim     322017 non-null  datetime64[ns]
dtypes:

In [14]:
df_viagem.head()

,id_viagem,Situação,Viagem Urgente,Código do órgão superior,Nome do órgão superior,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim
1,16332319,nao realizada,nao,26000,ministerio da educacao,***.211.458-**,claudia correa de andrade oliveira,professor do magisterio superior,sem informacao,sem informacao,2020-01-05,2021-01-04
2,16386658,realizada,nao,26000,ministerio da educacao,***.708.864-**,alexsandro galeno araujo dantas,professor do magisterio superior,fuc-0001,fuc-0001,2020-03-10,2020-03-12
3,16401038,realizada,nao,26000,ministerio da educacao,***.634.280-**,fernando carlos giacomelli,professor do magisterio superior,sem informacao,sem informacao,2020-01-01,2020-09-15
4,16413295,realizada,nao,26000,ministerio da educacao,***.464.507-**,rodrigo ramos hospodar felippe valverde,sem informacao,sem informacao,sem informacao,2020-03-02,2020-03-02
5,16415470,realizada,sim,26000,ministerio da educacao,***.126.127-**,martius vicente rodriguez y rodriguez,professor do magisterio superior,cd-0003,cargo comissionado de direcao,2020-01-11,2020-01-23


## df_passagem

In [15]:
df_passagem = pd.read_csv('./dataset/2020_Passagem.csv', delimiter=";", encoding='latin1')

In [16]:
df_passagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108981 entries, 0 to 108980
Data columns (total 19 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   Identificador do processo de viagem  108981 non-null  int64 
 1   Número da Proposta (PCDP)            108981 non-null  object
 2   Meio de transporte                   108981 non-null  object
 3   País - Origem ida                    108981 non-null  object
 4   UF - Origem ida                      106597 non-null  object
 5   Cidade - Origem ida                  108981 non-null  object
 6   País - Destino ida                   108981 non-null  object
 7   UF - Destino ida                     106058 non-null  object
 8   Cidade - Destino ida                 108981 non-null  object
 9   País - Origem volta                  108981 non-null  object
 10  UF - Origem volta                    108149 non-null  object
 11  Cidade - Origem volta     

In [17]:
df_passagem['Valor da passagem'] = df_passagem['Valor da passagem'].apply(lambda x: float(x.replace(',','.')))
df_passagem['Taxa de serviço'] = df_passagem['Taxa de serviço'].apply(lambda x: float(x.replace(',','.')))
df_passagem['Total'] = df_passagem['Valor da passagem'] + df_passagem['Taxa de serviço']

In [18]:
df_passagem = df_passagem.rename(columns={'Identificador do processo de viagem': 'id_viagem'})

In [19]:
df_passagem = df_passagem[~df_passagem['id_viagem'].isin(lista_id_remove)]

In [20]:
df_passagem = df_passagem.drop('Hora da emissão/compra', axis=1)
df_passagem = df_passagem.drop('Número da Proposta (PCDP)', axis=1)
df_passagem = df_passagem.drop('Data da emissão/compra', axis=1)
df_passagem = df_passagem.reset_index(drop=True)

In [21]:
df_passagem.index += 1
df_passagem['id_passagem'] = df_passagem.index

In [22]:
df_passagem = df_passagem.replace({np.nan: "Sem informação"})

In [23]:
# Tratamento de texto: acentos e simbolos, lower case e strip
for column in df_passagem.columns:
    if df_passagem[column].dtypes == 'object':
        df_passagem[column] = df_passagem[column].apply(clean_text)

In [24]:
df_passagem = df_passagem.replace({"nao informado": "sem informacao"})

In [25]:
df_passagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107015 entries, 1 to 107015
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id_viagem               107015 non-null  int64  
 1   Meio de transporte      107015 non-null  object 
 2   País - Origem ida       107015 non-null  object 
 3   UF - Origem ida         107015 non-null  object 
 4   Cidade - Origem ida     107015 non-null  object 
 5   País - Destino ida      107015 non-null  object 
 6   UF - Destino ida        107015 non-null  object 
 7   Cidade - Destino ida    107015 non-null  object 
 8   País - Origem volta     107015 non-null  object 
 9   UF - Origem volta       107015 non-null  object 
 10  Cidade - Origem volta   107015 non-null  object 
 11  Pais - Destino volta    107015 non-null  object 
 12  UF - Destino volta      107015 non-null  object 
 13  Cidade - Destino volta  107015 non-null  object 
 14  Valor da passagem   

In [26]:
df_passagem.head()

,id_viagem,Meio de transporte,País - Origem ida,UF - Origem ida,Cidade - Origem ida,País - Destino ida,UF - Destino ida,Cidade - Destino ida,País - Origem volta,UF - Origem volta,Cidade - Origem volta,Pais - Destino volta,UF - Destino volta,Cidade - Destino volta,Valor da passagem,Taxa de serviço,Total,id_passagem
1,16386658,aereo,brasil,rio grande do norte,natal,brasil,distrito federal,brasilia,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,522.04,1.14,523.18,1
2,16386658,aereo,brasil,distrito federal,brasilia,brasil,rio grande do norte,natal,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,529.57,1.14,530.71,2
3,16413295,aereo,brasil,sao paulo,sao paulo,brasil,distrito federal,brasilia,brasil,distrito federal,brasilia,brasil,sao paulo,sao paulo,627.42,1.14,628.56,3
4,16415470,aereo,brasil,rio de janeiro,rio de janeiro,portugal,sem informacao,lisboa,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,3451.84,0.00,3451.84,4
5,16415470,aereo,portugal,sem informacao,lisboa,brasil,rio de janeiro,rio de janeiro,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,3451.84,0.00,3451.84,5


## df_pag

In [27]:
df_pag = pd.read_csv('./dataset/2020_Pagamento.csv', delimiter=";", encoding='latin1')

In [28]:
df_pag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510770 entries, 0 to 510769
Data columns (total 10 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   Identificador do processo de viagem  510770 non-null  int64 
 1   Número da Proposta (PCDP)            510770 non-null  object
 2   Código do órgão superior             510770 non-null  int64 
 3   Nome do órgão superior               510770 non-null  object
 4   Codigo do órgão pagador              510770 non-null  int64 
 5   Nome do órgao pagador                398522 non-null  object
 6   Código da unidade gestora pagadora   510770 non-null  int64 
 7   Nome da unidade gestora pagadora     510770 non-null  object
 8   Tipo de pagamento                    510770 non-null  object
 9   Valor                                510770 non-null  object
dtypes: int64(4), object(6)
memory usage: 39.0+ MB


In [29]:
df_pag = df_pag[df_pag['Tipo de pagamento'] == 'PASSAGEM']

In [30]:
df_pag['Valor'] = df_pag['Valor'].apply(lambda x: float(x.replace(',','.')))

In [31]:
df_pag = df_pag.rename(columns={'Identificador do processo de viagem': 'id_viagem'})

In [32]:
df_pag = df_pag[~df_pag['id_viagem'].isin(lista_id_remove)]

In [33]:
df_pag = df_pag.drop('Número da Proposta (PCDP)', axis=1)
df_pag = df_pag.drop('Codigo do órgão pagador', axis=1)
df_pag = df_pag.drop('Nome do órgao pagador', axis=1)
df_pag = df_pag.drop('Código da unidade gestora pagadora', axis=1)
df_pag = df_pag.drop('Nome da unidade gestora pagadora', axis=1)
df_pag = df_pag.drop('Código do órgão superior', axis=1)
df_pag = df_pag.drop('Nome do órgão superior', axis=1)
df_pag = df_pag.drop('Tipo de pagamento', axis=1)
df_pag = df_pag.reset_index(drop=True)

In [34]:
df_pag = df_pag.replace({np.nan: "Sem informação"})

In [35]:
# Tratamento de texto: acentos e simbolos, lower case e strip
for column in df_pag.columns:
    if df_pag[column].dtypes == 'object':
        df_pag[column] = df_pag[column].apply(clean_text)

In [36]:
df_pag.index += 1

In [37]:
df_pag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132077 entries, 1 to 132077
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id_viagem  132077 non-null  int64  
 1   Valor      132077 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 2.0 MB


In [38]:
df_pag.head()

,id_viagem,Valor
1,16386658,523.18
2,16386658,530.71
3,16413295,628.56
4,16415470,3451.84
5,16415470,3451.84


# Analise e tabelas secundárias

## Cidade, UF, Pais

In [39]:
df_puc = df_passagem[['País - Origem ida', 'UF - Origem ida',
       'Cidade - Origem ida', 'País - Destino ida', 'UF - Destino ida',
       'Cidade - Destino ida', 'País - Origem volta', 'UF - Origem volta',
       'Cidade - Origem volta', 'Pais - Destino volta', 'UF - Destino volta',
       'Cidade - Destino volta']]

In [40]:
paises = list(set(list(df_puc['País - Origem ida'].unique()) + list(df_puc['País - Destino ida'].unique()) + list(df_puc['País - Origem volta'].unique()) + list(df_puc['Pais - Destino volta'].unique())))

In [41]:
df_paises= pd.DataFrame(paises, columns=['nome'])
df_paises.index += 1
df_paises['id_paises'] = df_paises.index
df_paises

,nome,id_paises
1,bosnia-herzegovina,1
2,arabia saudita,2
3,belarus,3
4,zimbabue,4
5,chile,5
...,...,...
126,mexico,126
127,filipinas,127
128,hungria,128
129,italia,129


In [42]:
uf = list(set(list(df_puc['UF - Origem ida'].unique()) + list(df_puc['UF - Destino ida'].unique()) + list(df_puc['UF - Origem volta'].unique()) + list(df_puc['Pais - Destino volta'].unique())))

In [43]:
df_uf = pd.DataFrame(uf, columns=['nome'])
df_uf.index += 1
df_uf['id_uf'] = df_uf.index
df_uf

,nome,id_uf
1,pernambuco,1
2,arabia saudita,2
3,belarus,3
4,chile,4
5,nova zelandia,5
...,...,...
87,servia,87
88,india,88
89,espirito santo,89
90,mexico,90


In [44]:
cidades = list(set(list(df_puc['Cidade - Origem ida'].unique()) + list(df_puc['Cidade - Destino ida'].unique()) + list(df_puc['Cidade - Origem volta'].unique()) + list(df_puc['Pais - Destino volta'].unique())))

In [45]:
df_cidades = pd.DataFrame(cidades, columns=['nome'])
df_cidades.index += 1
df_cidades['id_cidade'] = df_cidades.index
df_cidades

,nome,id_cidade
1,genebra,1
2,arabia saudita,2
3,lusaca,3
4,lanzhou,4
5,juina,5
...,...,...
736,servia,736
737,palotina,737
738,liubliana,738
739,alicante,739


## Ministério

In [46]:
df_ministerios = df_viagem.copy()
df_ministerios = df_ministerios.filter(['Código do órgão superior', 'Nome do órgão superior'])

In [47]:
df_ministerios = df_ministerios.rename(columns=
                                       {'Código do órgão superior': 'id_ministerio', 
                                        'Nome do órgão superior': 'nome_ministerio'})

In [48]:
df_ministerios = df_ministerios.drop_duplicates(subset=['id_ministerio'])

In [49]:
df_ministerios.head()

,id_ministerio,nome_ministerio
1,26000,ministerio da educacao
6,52000,ministerio da defesa
8,36000,ministerio da saude
30,35000,ministerio das relacoes exteriores
51,30000,ministerio da justica e seguranca publica


## Pessoa

In [50]:
df_pessoa = df_viagem.copy()

In [51]:
df_pessoa = df_pessoa.filter(['CPF viajante', 'Nome', 'Cargo', 'Função', 'Descrição Função'])

In [52]:
df_pessoa.head()

,CPF viajante,Nome,Cargo,Função,Descrição Função
1,***.211.458-**,claudia correa de andrade oliveira,professor do magisterio superior,sem informacao,sem informacao
2,***.708.864-**,alexsandro galeno araujo dantas,professor do magisterio superior,fuc-0001,fuc-0001
3,***.634.280-**,fernando carlos giacomelli,professor do magisterio superior,sem informacao,sem informacao
4,***.464.507-**,rodrigo ramos hospodar felippe valverde,sem informacao,sem informacao,sem informacao
5,***.126.127-**,martius vicente rodriguez y rodriguez,professor do magisterio superior,cd-0003,cargo comissionado de direcao


In [53]:
df_pessoa.drop_duplicates(subset=['CPF viajante'], inplace=True)

In [54]:
df_pessoa = df_pessoa.reset_index(drop=True)

In [55]:
df_pessoa_final = df_pessoa[['Nome', 'CPF viajante']]

In [56]:
df_pessoa_final.index += 1
df_pessoa_final['id_pessoa'] = df_pessoa_final.index

<ipython-input-56-26d9a24bddf6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pessoa_final['id_pessoa'] = df_pessoa_final.index


In [57]:
df_pessoa_final

,Nome,CPF viajante,id_pessoa
1,claudia correa de andrade oliveira,***.211.458-**,1
2,alexsandro galeno araujo dantas,***.708.864-**,2
3,fernando carlos giacomelli,***.634.280-**,3
4,rodrigo ramos hospodar felippe valverde,***.464.507-**,4
5,martius vicente rodriguez y rodriguez,***.126.127-**,5
...,...,...,...
105758,rodrigo campelo crispim,***.922.261-**,105758
105759,vinicius pacheco barcelos,***.679.700-**,105759
105760,cecilia silva gontijo,***.906.656-**,105760
105761,robson linko nunes da silva,***.873.083-**,105761


## Cargo

In [58]:
df_cargo = df_viagem[['Cargo']]

In [59]:
df_cargo = df_cargo.drop_duplicates(subset=['Cargo'])

In [60]:
df_cargo = df_cargo.reset_index(drop=True)

In [61]:
df_cargo.index += 1
df_cargo['id_cargo'] = df_cargo.index

In [62]:
df_cargo

,Cargo,id_cargo
1,professor do magisterio superior,1
2,sem informacao,2
3,pesquisador em saude publica,3
4,professor ens basico tecn tecnologico,4
5,professor tit-livre magist superior,5
...,...,...
686,diretor servidor do banco central,686
687,tecnico do banco central,687
688,presidente do banco central,688
689,diretor do banco central,689


## Função

In [63]:
df_funcao = df_viagem[['Função', 'Descrição Função']]

In [64]:
df_funcao = df_funcao.drop_duplicates(subset=['Função'])

In [65]:
df_funcao = df_funcao.reset_index(drop=True)

In [66]:
df_funcao.index += 1
df_funcao['id_funcao'] = df_funcao.index

In [67]:
df_funcao

,Função,Descrição Função,id_funcao
1,sem informacao,sem informacao,1
2,fuc-0001,fuc-0001,2
3,cd-0003,cargo comissionado de direcao,3
4,ofsuperior,ofsuperior,4
5,of.general,oficiais generais,5
...,...,...,...
138,gts-003,gratificacao temporaria - sipam,138
139,das-1033,sem informacao,139
140,das-1032,sem informacao,140
141,nes-0032,cargos de natureza especial,141


## Meio de transporte

In [68]:
meio = df_passagem['Meio de transporte']

In [69]:
meio = list(meio.unique())

In [70]:
df_meio = pd.DataFrame(meio, columns=['nome'])
df_meio.index += 1

df_meio['id_meio'] = df_meio.index

In [71]:
df_meio

,nome,id_meio
1,aereo,1
2,rodoviario,2
3,fluvial,3
4,maritimo,4


# Tabelas centralizadoras

In [72]:
df_viagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322017 entries, 1 to 322017
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   id_viagem                 322017 non-null  object        
 1   Situação                  322017 non-null  object        
 2   Viagem Urgente            322017 non-null  object        
 3   Código do órgão superior  322017 non-null  int64         
 4   Nome do órgão superior    322017 non-null  object        
 5   CPF viajante              322017 non-null  object        
 6   Nome                      322017 non-null  object        
 7   Cargo                     322017 non-null  object        
 8   Função                    322017 non-null  object        
 9   Descrição Função          322017 non-null  object        
 10  Período - Data de início  322017 non-null  datetime64[ns]
 11  Período - Data de fim     322017 non-null  datetime64[ns]
dtypes:

In [73]:
df_passagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107015 entries, 1 to 107015
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id_viagem               107015 non-null  int64  
 1   Meio de transporte      107015 non-null  object 
 2   País - Origem ida       107015 non-null  object 
 3   UF - Origem ida         107015 non-null  object 
 4   Cidade - Origem ida     107015 non-null  object 
 5   País - Destino ida      107015 non-null  object 
 6   UF - Destino ida        107015 non-null  object 
 7   Cidade - Destino ida    107015 non-null  object 
 8   País - Origem volta     107015 non-null  object 
 9   UF - Origem volta       107015 non-null  object 
 10  Cidade - Origem volta   107015 non-null  object 
 11  Pais - Destino volta    107015 non-null  object 
 12  UF - Destino volta      107015 non-null  object 
 13  Cidade - Destino volta  107015 non-null  object 
 14  Valor da passagem   

In [74]:
df_pag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132077 entries, 1 to 132077
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id_viagem  132077 non-null  int64  
 1   Valor      132077 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 2.0 MB


## pagamento_table

In [75]:
df_pag_final = df_pag.copy()
df_pag_final

,id_viagem,Valor
1,16386658,523.18
2,16386658,530.71
3,16413295,628.56
4,16415470,3451.84
5,16415470,3451.84
...,...,...
132073,2020000670,3088.48
132074,2020000671,1934.81
132075,2020000672,558.47
132076,2020000704,4148.92


In [76]:
df_passagem_cp = df_passagem.copy()

In [77]:
df_pag_final = pd.merge(df_passagem_cp, df_pag_final, how='left', right_on=['id_viagem','Valor'], left_on=['id_viagem','Total'])

In [78]:
df_pag_final = df_pag_final[['id_passagem', 'Taxa de serviço', 'Valor da passagem']]

In [79]:
df_pag_final.index +=1

In [80]:
df_pag_final['id_pagamento'] = df_pag_final.index

In [81]:
df_pag_final

,id_passagem,Taxa de serviço,Valor da passagem,id_pagamento
1,1,1.14,522.04,1
2,2,1.14,529.57,2
3,3,1.14,627.42,3
4,4,0.00,3451.84,4
5,4,0.00,3451.84,5
...,...,...,...,...
110589,107011,0.00,1084.34,110589
110590,107012,0.00,558.47,110590
110591,107013,0.00,1791.07,110591
110592,107014,0.00,2357.85,110592


## PUC_table

### Substituição passagem table

In [82]:
df_passagem_final = df_passagem.copy()

In [83]:
# Subs pelo index de cada p,u,c
for i, row in df_passagem_final.iterrows():
    df_passagem_final.at[i, 'País - Origem ida'] = paises.index(row['País - Origem ida']) + 1
    df_passagem_final.at[i, 'País - Destino ida'] = paises.index(row['País - Destino ida']) + 1
    df_passagem_final.at[i, 'País - Origem volta'] = paises.index(row['País - Origem volta']) + 1
    df_passagem_final.at[i, 'Pais - Destino volta'] = paises.index(row['Pais - Destino volta']) + 1
    
    df_passagem_final.at[i, 'UF - Origem ida'] = uf.index(row['UF - Origem ida']) + 1
    df_passagem_final.at[i, 'UF - Destino ida'] = uf.index(row['UF - Destino ida']) + 1
    df_passagem_final.at[i, 'UF - Origem volta'] = uf.index(row['UF - Origem volta']) + 1
    df_passagem_final.at[i, 'UF - Destino volta'] = uf.index(row['UF - Destino volta']) + 1
    
    df_passagem_final.at[i, 'Cidade - Origem ida'] = cidades.index(row['Cidade - Origem ida']) + 1
    df_passagem_final.at[i, 'Cidade - Destino ida'] = cidades.index(row['Cidade - Destino ida']) + 1
    df_passagem_final.at[i, 'Cidade - Origem volta'] = cidades.index(row['Cidade - Origem volta']) + 1
    df_passagem_final.at[i, 'Cidade - Destino volta'] = cidades.index(row['Cidade - Destino volta']) + 1

In [84]:
df_passagem_final

,id_viagem,Meio de transporte,País - Origem ida,UF - Origem ida,Cidade - Origem ida,País - Destino ida,UF - Destino ida,Cidade - Destino ida,País - Origem volta,UF - Origem volta,Cidade - Origem volta,Pais - Destino volta,UF - Destino volta,Cidade - Destino volta,Valor da passagem,Taxa de serviço,Total,id_passagem
1,16386658,aereo,63,54,462,63,76,256,118,83,355,118,83,355,522.04,1.14,523.18,1
2,16386658,aereo,63,76,256,63,54,462,118,83,355,118,83,355,529.57,1.14,530.71,2
3,16413295,aereo,63,16,29,63,76,256,63,76,256,63,16,29,627.42,1.14,628.56,3
4,16415470,aereo,63,40,466,57,83,490,118,83,355,118,83,355,3451.84,0.00,3451.84,4
5,16415470,aereo,57,83,490,63,40,466,118,83,355,118,83,355,3451.84,0.00,3451.84,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107011,2020000671,aereo,63,16,29,63,78,595,118,83,355,118,83,355,1084.34,0.00,1084.34,107011
107012,2020000672,aereo,63,76,256,63,16,29,118,83,355,118,83,355,558.47,0.00,558.47,107012
107013,2020000704,aereo,63,73,639,63,13,299,118,83,355,118,83,355,1791.07,0.00,1791.07,107013
107014,2020000704,aereo,63,13,299,63,73,171,118,83,355,118,83,355,2357.85,0.00,2357.85,107014


### puc table

In [85]:
# Criação lista puc
puc_lista = []

for i, row in df_passagem_final.iterrows():
    puc_origem_ida = (row['País - Origem ida'], row['UF - Origem ida'], row['Cidade - Origem ida'])
    puc_destino_ida = (row['País - Destino ida'], row['UF - Destino ida'], row['Cidade - Destino ida'])
    puc_origem_volta = (row['País - Origem volta'], row['UF - Origem volta'], row['Cidade - Origem volta'])
    puc_destino_volta = (row['Pais - Destino volta'], row['UF - Destino volta'], row['Cidade - Destino volta'])
    
    puc_lista.append(puc_origem_ida)
    puc_lista.append(puc_destino_ida)
    puc_lista.append(puc_origem_volta)
    puc_lista.append(puc_destino_volta)

In [86]:
df_puc_final = pd.DataFrame(puc_lista, columns=['id_pais', 'id_uf', 'id_cidade'])

In [87]:
df_puc_final = df_puc_final.drop_duplicates()
df_puc_final = df_puc_final.reset_index(drop=True)

In [88]:
df_puc_final.index +=1
df_puc_final['id_puc'] = df_puc_final.index

In [89]:
df_puc_final

,id_pais,id_uf,id_cidade,id_puc
1,63,54,462,1
2,63,76,256,2
3,118,83,355,3
4,63,16,29,4
5,63,40,466,5
...,...,...,...,...
691,63,84,51,691
692,63,19,658,692
693,126,83,240,693
694,17,11,197,694


## CPF_table

In [90]:
df_viagem_final = df_viagem.copy()

### Substituição viagem table

In [91]:
# Subs pelo index de cada c,p,f
pessoas = list(df_pessoa_final['CPF viajante'])
cargos = list(df_cargo['Cargo'])
funcoes = list(df_funcao['Função'])

for i, row in df_viagem_final.iterrows():
    df_viagem_final.at[i, 'CPF viajante'] = pessoas.index(row['CPF viajante']) + 1
    df_viagem_final.at[i, 'Cargo'] = cargos.index(row['Cargo']) + 1
    df_viagem_final.at[i, 'Função'] = funcoes.index(row['Função']) + 1

In [92]:
df_viagem_final

,id_viagem,Situação,Viagem Urgente,Código do órgão superior,Nome do órgão superior,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim
1,16332319,nao realizada,nao,26000,ministerio da educacao,1,claudia correa de andrade oliveira,1,1,sem informacao,2020-01-05,2021-01-04
2,16386658,realizada,nao,26000,ministerio da educacao,2,alexsandro galeno araujo dantas,1,2,fuc-0001,2020-03-10,2020-03-12
3,16401038,realizada,nao,26000,ministerio da educacao,3,fernando carlos giacomelli,1,1,sem informacao,2020-01-01,2020-09-15
4,16413295,realizada,nao,26000,ministerio da educacao,4,rodrigo ramos hospodar felippe valverde,2,1,sem informacao,2020-03-02,2020-03-02
5,16415470,realizada,sim,26000,ministerio da educacao,5,martius vicente rodriguez y rodriguez,1,3,cargo comissionado de direcao,2020-01-11,2020-01-23
...,...,...,...,...,...,...,...,...,...,...,...,...
322013,2020000671,realizada,nao,25000,ministerio da economia,105762,antonio carlos pereira rosa,687,1,sem informacao,2020-11-30,2020-12-04
322014,2020000672,realizada,nao,25000,ministerio da economia,105574,joao manoel pinho de mello,689,1,sem informacao,2020-12-10,2020-12-17
322015,2020000704,realizada,nao,25000,ministerio da economia,66331,juliano elizardo da silva,687,1,sem informacao,2020-12-14,2020-12-18
322016,2020000712,realizada,nao,25000,ministerio da economia,105605,fabio kanczuk,689,1,sem informacao,2020-12-17,2020-12-18


### cpf table

In [93]:
# Criação lista cpf
cpf_lista = []

for i, row in df_viagem_final.iterrows():
    cpf = (row['CPF viajante'], row['Cargo'], row['Função'])
    cpf_lista.append(cpf)

In [94]:
df_cpf = pd.DataFrame(cpf_lista, columns=['id_pessoa', 'id_cargo', 'id_funcao'])

In [95]:
df_cpf = df_cpf.drop_duplicates()
df_cpf = df_cpf.reset_index(drop=True)

In [96]:
df_cpf.index +=1
df_cpf['id_cpf'] = df_cpf.index

In [97]:
df_cpf

,id_pessoa,id_cargo,id_funcao,id_cpf
1,1,1,1,1
2,2,1,2,2
3,3,1,1,3
4,4,2,1,4
5,5,1,3,5
...,...,...,...,...
113455,105759,685,1,113455
113456,105760,687,1,113456
113457,105761,687,1,113457
113458,105762,687,1,113458


## passagem_table

In [98]:
df_passagem_final_2 = df_passagem_final.copy()

### Meio de transporte merge

In [99]:
df_passagem_final_2 = pd.merge(df_passagem_final_2, df_meio, left_on='Meio de transporte', how='left',right_on='nome')

In [100]:
df_passagem_final_2 = df_passagem_final_2.drop('Total', axis=1)
df_passagem_final_2 = df_passagem_final_2.drop('Meio de transporte', axis=1)
df_passagem_final_2 = df_passagem_final_2.drop('Valor da passagem', axis=1)
df_passagem_final_2 = df_passagem_final_2.drop('Taxa de serviço', axis=1)
df_passagem_final_2 = df_passagem_final_2.drop('nome', axis=1)

In [101]:
df_passagem_final_2.index += 1

In [102]:
df_passagem_final_2

,id_viagem,País - Origem ida,UF - Origem ida,Cidade - Origem ida,País - Destino ida,UF - Destino ida,Cidade - Destino ida,País - Origem volta,UF - Origem volta,Cidade - Origem volta,Pais - Destino volta,UF - Destino volta,Cidade - Destino volta,id_passagem,id_meio
1,16386658,63,54,462,63,76,256,118,83,355,118,83,355,1,1
2,16386658,63,76,256,63,54,462,118,83,355,118,83,355,2,1
3,16413295,63,16,29,63,76,256,63,76,256,63,16,29,3,1
4,16415470,63,40,466,57,83,490,118,83,355,118,83,355,4,1
5,16415470,57,83,490,63,40,466,118,83,355,118,83,355,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107011,2020000671,63,16,29,63,78,595,118,83,355,118,83,355,107011,1
107012,2020000672,63,76,256,63,16,29,118,83,355,118,83,355,107012,1
107013,2020000704,63,73,639,63,13,299,118,83,355,118,83,355,107013,1
107014,2020000704,63,13,299,63,73,171,118,83,355,118,83,355,107014,1


### PUC merge

In [103]:
df_puc_final

,id_pais,id_uf,id_cidade,id_puc
1,63,54,462,1
2,63,76,256,2
3,118,83,355,3
4,63,16,29,4
5,63,40,466,5
...,...,...,...,...
691,63,84,51,691
692,63,19,658,692
693,126,83,240,693
694,17,11,197,694


In [104]:
df_puc_merge = df_passagem_final_2.copy()

In [105]:
# Origem ida
df_puc_merge = pd.merge(df_puc_merge, df_puc_final, 
                        left_on=['País - Origem ida', 'UF - Origem ida', 'Cidade - Origem ida'], 
                        right_on=['id_pais', 'id_uf', 'id_cidade'],
                        how='left')

df_puc_merge = df_puc_merge.rename(columns={'id_puc': 'id_puc_origem_ida'})

In [106]:
# Destino ida
df_puc_merge = pd.merge(df_puc_merge, df_puc_final, 
                        left_on=['País - Destino ida', 'UF - Destino ida', 'Cidade - Destino ida'], 
                        right_on=['id_pais', 'id_uf', 'id_cidade'],
                        how='left')

df_puc_merge = df_puc_merge.rename(columns={'id_puc': 'id_puc_destino_ida'})

In [107]:
# Origem volta
df_puc_merge = pd.merge(df_puc_merge, df_puc_final, 
                        left_on=['País - Origem volta', 'UF - Origem volta', 'Cidade - Origem volta'], 
                        right_on=['id_pais', 'id_uf', 'id_cidade'],
                        how='left')

df_puc_merge = df_puc_merge.rename(columns={'id_puc': 'id_puc_origem_volta'})

In [108]:
# Destino volta
df_puc_merge = pd.merge(df_puc_merge, df_puc_final, 
                        left_on=['Pais - Destino volta', 'UF - Destino volta', 'Cidade - Destino volta'], 
                        right_on=['id_pais', 'id_uf', 'id_cidade'],
                        how='left')

df_puc_merge = df_puc_merge.rename(columns={'id_puc': 'id_puc_destino_volta'})

In [109]:
df_puc_merge = df_puc_merge[['id_passagem', 
                             'id_viagem', 
                             'id_meio', 
                             'id_puc_origem_ida', 
                             'id_puc_destino_ida', 
                             'id_puc_origem_volta', 
                             'id_puc_destino_volta']]

In [110]:
df_puc_merge.index += 1

In [111]:
df_passagem_final = df_puc_merge
df_passagem_final

,id_passagem,id_viagem,id_meio,id_puc_origem_ida,id_puc_destino_ida,id_puc_origem_volta,id_puc_destino_volta
1,1,16386658,1,1,2,3,3
2,2,16386658,1,2,1,3,3
3,3,16413295,1,4,2,2,4
4,4,16415470,1,5,6,3,3
5,5,16415470,1,6,5,3,3
...,...,...,...,...,...,...,...
107011,107011,2020000671,1,4,59,3,3
107012,107012,2020000672,1,2,4,3,3
107013,107013,2020000704,1,87,7,3,3
107014,107014,2020000704,1,7,20,3,3


## viagem_table

In [112]:
df_viagem_final_2 = df_viagem_final.copy()

In [113]:
df_viagem_final_2

,id_viagem,Situação,Viagem Urgente,Código do órgão superior,Nome do órgão superior,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim
1,16332319,nao realizada,nao,26000,ministerio da educacao,1,claudia correa de andrade oliveira,1,1,sem informacao,2020-01-05,2021-01-04
2,16386658,realizada,nao,26000,ministerio da educacao,2,alexsandro galeno araujo dantas,1,2,fuc-0001,2020-03-10,2020-03-12
3,16401038,realizada,nao,26000,ministerio da educacao,3,fernando carlos giacomelli,1,1,sem informacao,2020-01-01,2020-09-15
4,16413295,realizada,nao,26000,ministerio da educacao,4,rodrigo ramos hospodar felippe valverde,2,1,sem informacao,2020-03-02,2020-03-02
5,16415470,realizada,sim,26000,ministerio da educacao,5,martius vicente rodriguez y rodriguez,1,3,cargo comissionado de direcao,2020-01-11,2020-01-23
...,...,...,...,...,...,...,...,...,...,...,...,...
322013,2020000671,realizada,nao,25000,ministerio da economia,105762,antonio carlos pereira rosa,687,1,sem informacao,2020-11-30,2020-12-04
322014,2020000672,realizada,nao,25000,ministerio da economia,105574,joao manoel pinho de mello,689,1,sem informacao,2020-12-10,2020-12-17
322015,2020000704,realizada,nao,25000,ministerio da economia,66331,juliano elizardo da silva,687,1,sem informacao,2020-12-14,2020-12-18
322016,2020000712,realizada,nao,25000,ministerio da economia,105605,fabio kanczuk,689,1,sem informacao,2020-12-17,2020-12-18


### CPF merge

In [114]:
# Origem ida
df_cpf_merge = pd.merge(df_viagem_final_2, df_cpf, 
                        left_on=['CPF viajante', 'Cargo', 'Função'], 
                        right_on=['id_pessoa', 'id_cargo', 'id_funcao'],
                        how='left')

In [115]:
df_cpf_merge.index += 1

In [116]:
df_cpf_merge

,id_viagem,Situação,Viagem Urgente,Código do órgão superior,Nome do órgão superior,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim,id_pessoa,id_cargo,id_funcao,id_cpf
1,16332319,nao realizada,nao,26000,ministerio da educacao,1,claudia correa de andrade oliveira,1,1,sem informacao,2020-01-05,2021-01-04,1,1,1,1
2,16386658,realizada,nao,26000,ministerio da educacao,2,alexsandro galeno araujo dantas,1,2,fuc-0001,2020-03-10,2020-03-12,2,1,2,2
3,16401038,realizada,nao,26000,ministerio da educacao,3,fernando carlos giacomelli,1,1,sem informacao,2020-01-01,2020-09-15,3,1,1,3
4,16413295,realizada,nao,26000,ministerio da educacao,4,rodrigo ramos hospodar felippe valverde,2,1,sem informacao,2020-03-02,2020-03-02,4,2,1,4
5,16415470,realizada,sim,26000,ministerio da educacao,5,martius vicente rodriguez y rodriguez,1,3,cargo comissionado de direcao,2020-01-11,2020-01-23,5,1,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322013,2020000671,realizada,nao,25000,ministerio da economia,105762,antonio carlos pereira rosa,687,1,sem informacao,2020-11-30,2020-12-04,105762,687,1,113458
322014,2020000672,realizada,nao,25000,ministerio da economia,105574,joao manoel pinho de mello,689,1,sem informacao,2020-12-10,2020-12-17,105574,689,1,113248
322015,2020000704,realizada,nao,25000,ministerio da economia,66331,juliano elizardo da silva,687,1,sem informacao,2020-12-14,2020-12-18,66331,687,1,113450
322016,2020000712,realizada,nao,25000,ministerio da economia,105605,fabio kanczuk,689,1,sem informacao,2020-12-17,2020-12-18,105605,689,1,113282


### Finalização

In [117]:
df_viagem_final = df_cpf_merge[['id_viagem', 
                             'Situação', 
                             'Viagem Urgente', 
                             'Período - Data de início', 
                             'Período - Data de fim', 
                             'Código do órgão superior',
                             'id_cpf']]

In [118]:
df_viagem_final = df_viagem_final.rename(columns={"Situação": 'is_realizado', 
                                                  'Viagem Urgente': 'is_urgente', 
                                                  'Período - Data de início': 'data_inicio', 
                                                  'Período - Data de fim': 'data_fim',
                                                  'Código do órgão superior': 'id_ministerio'})

In [119]:
df_viagem_final['is_realizado'] = df_viagem_final['is_realizado'].map({"nao realizada": False, "realizada": True})
df_viagem_final['is_urgente'] = df_viagem_final['is_urgente'].map({"nao": False, "sim": True})

In [120]:
df_viagem_final

,id_viagem,is_realizado,is_urgente,data_inicio,data_fim,id_ministerio,id_cpf
1,16332319,False,False,2020-01-05,2021-01-04,26000,1
2,16386658,True,False,2020-03-10,2020-03-12,26000,2
3,16401038,True,False,2020-01-01,2020-09-15,26000,3
4,16413295,True,False,2020-03-02,2020-03-02,26000,4
5,16415470,True,True,2020-01-11,2020-01-23,26000,5
...,...,...,...,...,...,...,...
322013,2020000671,True,False,2020-11-30,2020-12-04,25000,113458
322014,2020000672,True,False,2020-12-10,2020-12-17,25000,113248
322015,2020000704,True,False,2020-12-14,2020-12-18,25000,113450
322016,2020000712,True,False,2020-12-17,2020-12-18,25000,113282


# Resumo das tabelas

## Viagem, passagem e pagamento

In [121]:
df_viagem_final

,id_viagem,is_realizado,is_urgente,data_inicio,data_fim,id_ministerio,id_cpf
1,16332319,False,False,2020-01-05,2021-01-04,26000,1
2,16386658,True,False,2020-03-10,2020-03-12,26000,2
3,16401038,True,False,2020-01-01,2020-09-15,26000,3
4,16413295,True,False,2020-03-02,2020-03-02,26000,4
5,16415470,True,True,2020-01-11,2020-01-23,26000,5
...,...,...,...,...,...,...,...
322013,2020000671,True,False,2020-11-30,2020-12-04,25000,113458
322014,2020000672,True,False,2020-12-10,2020-12-17,25000,113248
322015,2020000704,True,False,2020-12-14,2020-12-18,25000,113450
322016,2020000712,True,False,2020-12-17,2020-12-18,25000,113282


In [122]:
df_passagem_final

,id_passagem,id_viagem,id_meio,id_puc_origem_ida,id_puc_destino_ida,id_puc_origem_volta,id_puc_destino_volta
1,1,16386658,1,1,2,3,3
2,2,16386658,1,2,1,3,3
3,3,16413295,1,4,2,2,4
4,4,16415470,1,5,6,3,3
5,5,16415470,1,6,5,3,3
...,...,...,...,...,...,...,...
107011,107011,2020000671,1,4,59,3,3
107012,107012,2020000672,1,2,4,3,3
107013,107013,2020000704,1,87,7,3,3
107014,107014,2020000704,1,7,20,3,3


In [123]:
df_pag_final = df_pag_final.rename(columns={'Taxa de serviço': 'taxa', 'Valor da passagem': 'valor',})
df_pag_final = df_pag_final[['id_pagamento', 'valor', 'taxa', 'id_passagem']]

In [124]:
df_pag_final

,id_pagamento,valor,taxa,id_passagem
1,1,522.04,1.14,1
2,2,529.57,1.14,2
3,3,627.42,1.14,3
4,4,3451.84,0.00,4
5,5,3451.84,0.00,4
...,...,...,...,...
110589,110589,1084.34,0.00,107011
110590,110590,558.47,0.00,107012
110591,110591,1791.07,0.00,107013
110592,110592,2357.85,0.00,107014


## CPF e PUC

In [125]:
df_cpf = df_cpf[['id_cpf', 'id_pessoa', 'id_cargo', 'id_funcao']]

In [126]:
df_cpf

,id_cpf,id_pessoa,id_cargo,id_funcao
1,1,1,1,1
2,2,2,1,2
3,3,3,1,1
4,4,4,2,1
5,5,5,1,3
...,...,...,...,...
113455,113455,105759,685,1
113456,113456,105760,687,1
113457,113457,105761,687,1
113458,113458,105762,687,1


In [127]:
df_puc_final = df_puc_final[['id_puc', 'id_pais', 'id_uf', 'id_cidade',]]

In [128]:
df_puc_final

,id_puc,id_pais,id_uf,id_cidade
1,1,63,54,462
2,2,63,76,256
3,3,118,83,355
4,4,63,16,29
5,5,63,40,466
...,...,...,...,...
691,691,63,84,51
692,692,63,19,658
693,693,126,83,240
694,694,17,11,197


## Resto

In [129]:
df_paises = df_paises[['id_paises', 'nome']]

In [130]:
df_paises

,id_paises,nome
1,1,bosnia-herzegovina
2,2,arabia saudita
3,3,belarus
4,4,zimbabue
5,5,chile
...,...,...
126,126,mexico
127,127,filipinas
128,128,hungria
129,129,italia


In [131]:
df_uf = df_uf[['id_uf', 'nome']]

In [132]:
df_uf

,id_uf,nome
1,1,pernambuco
2,2,arabia saudita
3,3,belarus
4,4,chile
5,5,nova zelandia
...,...,...
87,87,servia
88,88,india
89,89,espirito santo
90,90,mexico


In [133]:
df_cidades = df_cidades[['id_cidade', 'nome']]

In [134]:
df_cidades

,id_cidade,nome
1,1,genebra
2,2,arabia saudita
3,3,lusaca
4,4,lanzhou
5,5,juina
...,...,...
736,736,servia
737,737,palotina
738,738,liubliana
739,739,alicante


In [135]:
df_pessoa_final = df_pessoa_final.rename(columns={'Nome': 'nome', 'CPF viajante': 'cpf',})
df_pessoa_final = df_pessoa_final[['id_pessoa', 'nome', 'cpf']]

In [136]:
df_pessoa_final

,id_pessoa,nome,cpf
1,1,claudia correa de andrade oliveira,***.211.458-**
2,2,alexsandro galeno araujo dantas,***.708.864-**
3,3,fernando carlos giacomelli,***.634.280-**
4,4,rodrigo ramos hospodar felippe valverde,***.464.507-**
5,5,martius vicente rodriguez y rodriguez,***.126.127-**
...,...,...,...
105758,105758,rodrigo campelo crispim,***.922.261-**
105759,105759,vinicius pacheco barcelos,***.679.700-**
105760,105760,cecilia silva gontijo,***.906.656-**
105761,105761,robson linko nunes da silva,***.873.083-**


In [137]:
df_cargo = df_cargo.rename(columns={'Cargo': 'nome', })
df_cargo = df_cargo[['id_cargo', 'nome']]

In [138]:
df_cargo

,id_cargo,nome
1,1,professor do magisterio superior
2,2,sem informacao
3,3,pesquisador em saude publica
4,4,professor ens basico tecn tecnologico
5,5,professor tit-livre magist superior
...,...,...
686,686,diretor servidor do banco central
687,687,tecnico do banco central
688,688,presidente do banco central
689,689,diretor do banco central


In [139]:
df_funcao = df_funcao.rename(columns={'Descrição Função': 'descricao', 'Função': 'nome'})
df_funcao = df_funcao[['id_funcao', 'nome', 'descricao']]

In [140]:
df_funcao

,id_funcao,nome,descricao
1,1,sem informacao,sem informacao
2,2,fuc-0001,fuc-0001
3,3,cd-0003,cargo comissionado de direcao
4,4,ofsuperior,ofsuperior
5,5,of.general,oficiais generais
...,...,...,...
138,138,gts-003,gratificacao temporaria - sipam
139,139,das-1033,sem informacao
140,140,das-1032,sem informacao
141,141,nes-0032,cargos de natureza especial


In [141]:
df_ministerios = df_ministerios.rename(columns={'nome_ministerio': 'nome'})

In [142]:
df_ministerios

,id_ministerio,nome
1,26000,ministerio da educacao
6,52000,ministerio da defesa
8,36000,ministerio da saude
30,35000,ministerio das relacoes exteriores
51,30000,ministerio da justica e seguranca publica
200,25000,ministerio da economia
283,37000,controladoria-geral da uniao
292,39000,ministerio da infraestrutura
330,32000,ministerio de minas e energia
369,-1,sem informacao


In [143]:
df_meio = df_meio[['id_meio', 'nome']]

In [144]:
df_meio.head()

,id_meio,nome
1,1,aereo
2,2,rodoviario
3,3,fluvial
4,4,maritimo


# Salvando

In [145]:
df_viagem_final.to_csv('./dataset/gerados/viagem.csv')
df_passagem_final.to_csv('./dataset/gerados/passagem.csv')
df_pag_final.to_csv('./dataset/gerados/pagamento.csv')
df_cpf.to_csv('./dataset/gerados/cpf.csv')
df_puc_final.to_csv('./dataset/gerados/puc.csv')
df_paises.to_csv('./dataset/gerados/paises.csv')
df_uf.to_csv('./dataset/gerados/uf.csv')
df_cidades.to_csv('./dataset/gerados/cidades.csv')
df_pessoa_final.to_csv('./dataset/gerados/pessoa.csv')
df_cargo.to_csv('./dataset/gerados/cargo.csv')
df_funcao.to_csv('./dataset/gerados/funcao.csv')
df_ministerios.to_csv('./dataset/gerados/ministerio.csv')
df_meio.to_csv('./dataset/gerados/meio.csv')